# Câmara dos Deputados - Cota Parlamentar 2015

Essa é uma análise da utilização da Cota Parlamentar dos Deputados da Câmara Federal no ano de 2015. Os dados estão disponíveis no site da Câmara em formato XML. Faça o download.

Um arquivo XML nada mais é do que um arquivo texto que utiliza uma linguagem de marcadores que se assemelha muito com o HTML. Enquanto o HTML possui marcadores padrão, definidos pela W3C, o XML pode apresentar marcadores arbitrários, criados por quem gera o arquivo.

Existem diversas formas de trabalhar com arquivos XML. Contudo, nesse optamos por converter o arquivo XML para CSV por alguns motivos:

- O XML gerado pelo sistema da Câmara pode ser representado de forma bi-dimensional na forma linhas vs. colunas;
- O arquivo CSV convertido é 10 vezes menor que o XML, facilitando o processamento e redistribuição na rede;
- A maior parte das ferramentas de análise de dados conseguem trabalhar com CSV. O contrário não é verdade.

Utilizamos o módulo ```xml.etree.ElementTree``` do Python para converter o arquivo ```XML``` em ```CSV```. Você precisará de, pelo menos, 7.5GB de memória RAM livres para fazer a conversão. Tanta memória é necessária por causa de um atributo inexplicável do arquivo da Câmara.

## XML :(
Toda a informação do arquivo XML foi registrada em apenas **uma linha** no arquivo. Como a conversão acontece linha por linha não há outra alternativa a não ser carregar a primeira e única linha do arquivo, que contém todos os milhares (em alguns casos milhões) de registros para a memória do computador. Isso seria facilmente contornável se o sistema da Câmara tivesse formatado o ```XML``` em linhas diferentes. Ou melhor ainda, se eles tivessem seguido o padrão do Senado e do Governo Federal disponibilizando arquivos no formato ```CSV```.

## A estrutura do arquivo XML da câmara
O arquivo XML da câmara possui a seguinte estrutura:

```<DESPESAS>
        <DESPESA>
            <txNomeParlamentar>VALOR</txNomeParlamentar>
            <ideCadastro>VALOR</ideCadastro>
            <nuCarteiraParlamentar>VALOR</nuCarteiraParlamentar>
            .
            .
            .
            <nuDeputadoId>VALOR</nuDeputadoId>
         </DESPESA>
         <DESPESA>
            <txNomeParlamentar>VALOR</txNomeParlamentar>
            <ideCadastro>VALOR</ideCadastro>
            <nuCarteiraParlamentar>VALOR</nuCarteiraParlamentar>
            .
            .
            .
            <nuDeputadoId>VALOR</nuDeputadoId>
         </DESPESA>
         .
         .
         .
         <DESPESA>
            <txNomeParlamentar>VALOR</txNomeParlamentar>
            <ideCadastro>VALOR</ideCadastro>
            <nuCarteiraParlamentar>VALOR</nuCarteiraParlamentar>
            .
            .
            .
            <nuDeputadoId>VALOR</nuDeputadoId>
         </DESPESA>
   </DESPESAS>```

Cada bloco ```<DESPESA></DESPESA>``` é o equivalente a um registro (linha) no arquivo ```CSV```. Dentro do bloco <DESPESA></DESPESA> existem 28 marcadores, ou tags, que são equivalentes às colunas no arquivo ```CSV```, com seus respectivos valores.

Com essas informações podemos construir o arquivo ```CSV``` usando a seguinte lógica:

- Serão 28 colunas no total
- As colunas receberão os **nomes** dos marcadores-filhos do marcador ```<DESPESA>```
- Cada linha será populada com os 28 **valores** dos marcadores-filhos de ```<DESPESA>```

## Carregando o XML
Primeiro vamos definir a codificação que vamos trabalhar, no caso, UTF-8. Em seguida, importamos os módulos ```XML``` e ```CSV```.

In [2]:
# -*- coding: utf-8
import xml.etree.ElementTree as ET
import csv

Vamos configurar algumas variávies:

In [ ]:
csv_arquivo = 'deputados_2015_revisado.csv' # Guarda o nome do arquivo CSV que vamos criar
xml_arquivo = 'deputados_2015.xml' # Guarda o nome do arquivo XML com os dados da Câmara
tree = ET.parse(xml_arquivo) # Carrega o XML para a memória do computador interpretando sua estrutura
root = tree.getroot() # Carrega a lista de objetos na raíz do XML
root = root[0] # Carrega a lista de todos os blocos <DESPESA> que estão na raíz

O site da Câmara disponibiliza os 28 nomes dos marcadores e a descrição de cada um deles. Esses nomes serão as nossas coliunas. Vamos guardá-la numa lista para usarmos depois. Também é possível descobrir os nomes dos marcadores fazendo chamadas ao objeto ```root```.

In [ ]:
header_row = ["txNomeParlamentar","ideCadastro","nuCarteiraParlamentar",\
              "nuLegislatura","sgUF","sgPartido","codLegislatura","numSubCota",\
              "txtDescricao","numEspecificacaoSubCota","txtDescricaoEspecificacao",\
              "txtFornecedor","txtCNPJCPF","txtNumero","indTipoDocumento","datEmissao",\
              "vlrDocumento","vlrGlosa","vlrLiquido","numMes","numAno","numParcela",\
              "txtPassageiro","txtTrecho","numLote","numRessarcimento", "vlrRestituicao",\
              "nuDeputadoId"]

O bloco seguinte itera sobre os blocos de despesa salva as informações de marcador-filho lista de valores. Esses valores chamada ```complete_row```. O contador serve para verificar se estamos na primeira linha para registro dos nomes das colunas (```header_row```).

In [ ]:
counter = 0 # Contador para registrar primeira linha
with open(csv_arquivo, 'w+') as csv_file:
    csv_writer = csv.writer(csv_file)
    for row in root:
        complete_row = [] # Lista de valores da linha corrente
        for value in row:
            try:
                complete_row.append(value.text.encode('utf-8')) # Acrescenta o valor do bloco na lista de valores
            except:
                try:
                    complete_row.append(value.text)
                except:
                    print "Coulnd't write value", value
                    pass
        if counter is 0: # Se o contador for 0, ou seja, se estivermos na primeira linha
            csv_writer.writerow(header_row) # Registrar cabeçalho (nomes das colunas) no arquivo CSV
            csv_writer.writerow(complete_row) # Registrar também valores da linha
        else:
            csv_writer.writerow(complete_row) # Caso contrário, apenas registrar valores da linha
        counter += 1
        print "Linha", counter, "salva com sucesso!"
    csv_file.close() # Fecha arquivo CSV

Os passos acima são suficientes para gerar o arquivo ```CSV``` na mesma lógica estrutural do arquivo XML.

# Pandas

Com o arquivo ```CSV``` em mãos vamos usar a biblioteca Pandas para fazer consultas e gerar recortes da tabela principal. O Pandas, por ter motores desenvolvidos em ```C```, é capaz de fazer buscas e gerar visualizações com extrema rapidez ao mesmo tempo que não demanda muito do computador. Para importar o Pandas, basta:

In [1]:
import pandas

Em seguida, vamos criar um modelo de dados do Pandas a partir do arquivo ```CSV``` que geramos anteriormente. Vamos chamar esse modelo de base e vamos usar o método ```read_csv``` com os seguintes parâmetros:

- ```error_bad_lines=False # A geração do modelo não é interrompida caso alguma linha esteja corrompida```
- ```warn_bad_lines=True # O sistema avisa qual linha apresentou problema, se for o caso```
- ```low_memory=False # O arquivo da Câmara mistura tipos de dados numa mesma coluna (tsc, tsc). Ou você força a definição do tipo de dado usando o parâmetro dtype ou você desativa o parâmetro low_memory```

In [2]:
base = pandas.read_csv('data/deputados_2015_fornecedor_consolidado.csv', error_bad_lines=False, warn_bad_lines=True)

Modelo criado, podemos digitar ```base``` para visualizar rapidamente as primeiras e últimas linhas da base:

In [9]:
base

,nuDeputadoId,sgPartido,numMes,txtCNPJCPF,vlrGlosa,txNomeParlamentar,sgUF,nuCarteiraParlamentar,vlrDocumento,numSubCota,...,txtFornecedorConsolidado,numLote,indTipoDocumento,numAno,txtTrecho,numRessarcimento,vlrLiquido,ideDocumento,ideCadastro,nuLegislatura
0,3074,DEM,11,5.939467e+12,0.00,ABEL MESQUITA JR.,RR,1.0,165.65,1,...,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,1255355,0,2015,NaN,5294.0,165.65,5886345.0,178957.0,2015
1,3074,DEM,12,5.939467e+12,0.00,ABEL MESQUITA JR.,RR,1.0,59.48,1,...,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,1255361,0,2015,NaN,5294.0,59.48,5886361.0,178957.0,2015
2,3074,DEM,11,2.341470e+12,0.00,ABEL MESQUITA JR.,RR,1.0,130.95,1,...,ELETROBRAS DISTRIBUIÇÃO RORAIMA,1255355,0,2015,NaN,5294.0,130.95,5886341.0,178957.0,2015
3,3074,DEM,12,2.341470e+12,3.47,ABEL MESQUITA JR.,RR,1.0,196.53,1,...,ELETROBRAS DISTRIBUIÇÃO RORAIMA,1268867,0,2015,NaN,5370.0,193.06,5928783.0,178957.0,2015
4,3074,DEM,2,5.402520e+11,0.00,ABEL MESQUITA JR.,RR,1.0,310.25,1,...,PAPELARIA ABC COM. E IND. LTDA.,1168538,0,2015,NaN,4934.0,0.00,5608486.0,178957.0,2015
5,3074,DEM,2,5.402520e+11,0.00,ABEL MESQUITA JR.,RR,1.0,310.25,1,...,PAPELARIA ABC COM. E IND. LTDA.,1168538,0,2015,NaN,4966.0,310.25,5608486.0,178957.0,2015
6,3074,DEM,6,8.202116e+12,0.00,ABEL MESQUITA JR.,RR,1.0,32.00,3,...,AUTO POSTO AEROPORTO LTDA.,1220276,0,2015,NaN,5131.0,32.00,5772352.0,178957.0,2015
7,3074,DEM,8,8.202116e+12,0.00,ABEL MESQUITA JR.,RR,1.0,50.00,3,...,AUTO POSTO AEROPORTO LTDA.,1220275,0,2015,NaN,5131.0,50.00,5772246.0,178957.0,2015
8,3074,DEM,9,8.202116e+12,0.00,ABEL MESQUITA JR.,RR,1.0,50.00,3,...,AUTO POSTO AEROPORTO LTDA.,1229908,0,2015,NaN,5173.0,50.00,5804438.0,178957.0,2015
9,3074,DEM,9,8.202116e+12,0.00,ABEL MESQUITA JR.,RR,1.0,75.00,3,...,AUTO POSTO AEROPORTO LTDA.,1229908,0,2015,NaN,5173.0,75.00,5804312.0,178957.0,2015


A função ```head()``` permite visualizar rapidamente as 5 primeiras linhas da tabela: 

In [4]:
base.head()

,nuDeputadoId,sgPartido,numMes,txtCNPJCPF,vlrGlosa,txNomeParlamentar,sgUF,nuCarteiraParlamentar,vlrDocumento,numSubCota,...,txtFornecedorConsolidado,numLote,indTipoDocumento,numAno,txtTrecho,numRessarcimento,vlrLiquido,ideDocumento,ideCadastro,nuLegislatura
0,3074,DEM,11,5.939467e+12,0.00,ABEL MESQUITA JR.,RR,1.0,165.65,1,...,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,1255355,0,2015,NaN,5294.0,165.65,5886345.0,178957.0,2015
1,3074,DEM,12,5.939467e+12,0.00,ABEL MESQUITA JR.,RR,1.0,59.48,1,...,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,1255361,0,2015,NaN,5294.0,59.48,5886361.0,178957.0,2015
2,3074,DEM,11,2.341470e+12,0.00,ABEL MESQUITA JR.,RR,1.0,130.95,1,...,ELETROBRAS DISTRIBUIÇÃO RORAIMA,1255355,0,2015,NaN,5294.0,130.95,5886341.0,178957.0,2015
3,3074,DEM,12,2.341470e+12,3.47,ABEL MESQUITA JR.,RR,1.0,196.53,1,...,ELETROBRAS DISTRIBUIÇÃO RORAIMA,1268867,0,2015,NaN,5370.0,193.06,5928783.0,178957.0,2015
4,3074,DEM,2,5.402520e+11,0.00,ABEL MESQUITA JR.,RR,1.0,310.25,1,...,PAPELARIA ABC COM. E IND. LTDA.,1168538,0,2015,NaN,4934.0,0.00,5608486.0,178957.0,2015


# 1. Soma de gastos por parlamentar
### ...com referência ao estado e partido

O Pandas permite criar tabelas dinâmicas assim como a maioria dos processadores de planilhas eletrônicas. O parâmetro ```index``` organiza as lihas da tabela e pode receber uma lista de resultados. O parâmetro ```values``` associa valores com base numa função de agregamento (```aggfunc```). No nosso caso, estamos agregando os valores da coluna ```vlrDocumento``` com a função ```sum``` (somar). Para salvar um arquivo CSV dessa visualização, basta acrescentar ```.to_csv('nomedoarquivo.csv')``` ao fim do comando abaixo.

In [5]:
base.pivot_table(index=['txNomeParlamentar','sgUF', 'sgPartido'], values=['vlrDocumento'], aggfunc='sum')

,,,vlrDocumento
txNomeParlamentar,sgUF,sgPartido,
ABEL MESQUITA JR.,RR,DEM,507558.70
ABELARDO CAMARINHA,SP,PSB,33377.54
ABELARDO LUPION,PR,DEM,32848.85
ACELINO POPÓ,BA,PRB,34561.34
ADAIL CARNEIRO,CE,S.PART.,405115.89
ADALBERTO CAVALCANTI,PE,PTB,459389.79
ADELMO CARNEIRO LEÃO,MG,PT,412854.36
ADELSON BARRETO,SE,PR,285548.16
ADEMIR CAMILO,MG,PTN,518069.96


In [4]:
base.pivot_table(index=['txNomeParlamentar','sgUF', 'sgPartido'], values=['vlrDocumento'], aggfunc='sum').to_csv('ranking_deputados.csv')

In [8]:
base.loc[base['vlrDocumento'] > 10000]

,sgPartido,numMes,txtCNPJCPF,vlrGlosa,txNomeParlamentar,sgUF,nuCarteiraParlamentar,vlrDocumento,nuDeputadoId,numSubCota,...,txtFornecedor,numLote,indTipoDocumento,numAno,txtTrecho,numRessarcimento,vlrLiquido,ideDocumento,ideCadastro,nuLegislatura
84,DEM,11,1.687189e+13,0.0,ABEL MESQUITA JR.,RR,1.0,18000.0,3074,4,...,ASPAR CENTRO DE SERVIÇOS PARLAMENTARES E GESTÃ...,1244585,0,2015,NaN,5242.0,18000.00,5851959.0,178957.0,2015
85,DEM,12,1.687189e+13,0.0,ABEL MESQUITA JR.,RR,1.0,18000.0,3074,4,...,ASPAR CENTRO DE SERVIÇOS PARLAMENTARES E GESTÃ...,1255689,0,2015,NaN,5298.0,18000.00,5887522.0,178957.0,2015
86,DEM,7,1.692372e+13,0.0,ABEL MESQUITA JR.,RR,1.0,20000.0,3074,4,...,EFZ COMERCIO E REPRESENTAÇÃO LTDA,1217319,0,2015,NaN,5124.0,20000.00,5761747.0,178957.0,2015
87,DEM,8,1.692372e+13,0.0,ABEL MESQUITA JR.,RR,1.0,20000.0,3074,4,...,EFZ COMERCIO E REPRESENTAÇÃO LTDA,1226779,0,2015,NaN,5170.0,20000.00,5793677.0,178957.0,2015
88,DEM,10,1.692372e+13,0.0,ABEL MESQUITA JR.,RR,1.0,20000.0,3074,4,...,EFZ COMERCIO E REPRESENTAÇÃO LTDA,1249639,0,2015,NaN,5260.0,20000.00,5868694.0,178957.0,2015
89,DEM,12,1.692372e+13,0.0,ABEL MESQUITA JR.,RR,1.0,15000.0,3074,4,...,EFZ COMERCIO E REPRESENTAÇÃO LTDA,1255712,0,2015,NaN,5323.0,15000.00,5887581.0,178957.0,2015
90,DEM,9,9.021305e+12,0.0,ABEL MESQUITA JR.,RR,1.0,40000.0,3074,4,...,F. DA S. RABELO EPP,1244337,0,2015,NaN,5236.0,0.00,5850938.0,178957.0,2015
91,DEM,9,9.021305e+12,0.0,ABEL MESQUITA JR.,RR,1.0,40000.0,3074,4,...,F. DA S. RABELO EPP,1244337,0,2015,NaN,5238.0,40000.00,5850938.0,178957.0,2015
92,DEM,10,9.021305e+12,0.0,ABEL MESQUITA JR.,RR,1.0,40000.0,3074,4,...,F. DA S. RABELO EPP,1244340,0,2015,NaN,5238.0,40000.00,5850900.0,178957.0,2015
93,DEM,6,1.506344e+13,0.0,ABEL MESQUITA JR.,RR,1.0,30000.0,3074,5,...,UP COMÉRCIO E INDÚSTRIA GRÁFICA LTDA,1217323,0,2015,NaN,5124.0,30000.00,5761791.0,178957.0,2015


In [50]:
base.pivot_table(index='txtFornecedor', values='vlrDocumento', aggfunc='sum').to_csv('empresas.csv')

In [18]:
base.pivot_table(index=['txNomeParlamentar','sgUF', 'sgPartido', 'txtDescricao'], values=['vlrDocumento'], aggfunc='sum')

vlrDocumento
txNomeParlamentar  sgUF sgPartido txtDescricao                                                    
ABEL MESQUITA JR.  RR   DEM       COMBUSTÍVEIS E LUBRIFICANTES.                           23485.06
                                  CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.          231000.00
                                  DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                    80000.00
                                  EMISSÃO BILHETE AÉREO                                   75064.03
                                  FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                247.22
                                  LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES                     9500.00
                                  LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES           30240.90
                                  MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE P...       1173.11
                                  PASSAGENS AÉREAS                                         1926.98
                                  SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPEC...      42500.00
                                  SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                  21.70
                                  SERVIÇOS POSTAIS                                          257.15
                                  TELEFONIA                                               12142.55
ABELARDO CAMARINHA SP   PSB       COMBUSTÍVEIS E LUBRIFICANTES.                            2051.03
                                  DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                    30000.00
                                  HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO F...        283.00
                                  SERVIÇOS POSTAIS                                          432.40
                                  TELEFONIA                                                 611.11
ABELARDO LUPION    PR   DEM       DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                    29964.00
                                  EMISSÃO BILHETE AÉREO                                    2070.50
                                  SERVIÇOS POSTAIS                                          203.15
                                  TELEFONIA                                                 611.20
ACELINO POPÓ       BA   PRB       CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.           10500.00
                                  DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                    18000.00
                                  EMISSÃO BILHETE AÉREO                                    1820.57
                                  MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE P...       3071.20
                                  SERVIÇOS POSTAIS                                          969.80
                                  TELEFONIA                                                 199.77
ADAIL CARNEIRO     CE   S.PART.   COMBUSTÍVEIS E LUBRIFICANTES.                           36102.16
                                  CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.           65000.00
...                                                                                            ...
ZÉ SILVA           MG   SD        TELEFONIA                                               23168.14
ZÉ VIEIRA          MA   PROS      DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                    28000.00
                                  EMISSÃO BILHETE AÉREO                                    1728.13
                                  SERVIÇOS POSTAIS                                           57.20
                                  TELEFONIA                                                1297.44
ÁTILA LINS         AM   PSD       CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.           40000.00
                                  EMISSÃO BILHETE AÉREO                                   92680.33
                                  LOCAÇÃO OU FRETAMENTO DE AERONAVES                     292900.00
                           

In [14]:
base.pivot_table(index=['txNomeParlamentar','sgUF', 'sgPartido', 'txtDescricao'], values=['vlrDocumento'], aggfunc='sum').to_csv('deputados.csv')

In [19]:
base.pivot_table(index=['txNomeParlamentar','sgUF', 'sgPartido', 'txtDescricao', 'txtFornecedorConsolidado'], values=['vlrDocumento'], aggfunc='sum')

vlrDocumento
txNomeParlamentar sgUF sgPartido txtDescricao                                       txtFornecedorConsolidado                                        
ABEL MESQUITA JR. RR   DEM       COMBUSTÍVEIS E LUBRIFICANTES.                      AUTO POSTO AEROPORTO LTDA.                                207.00
                                                                                    AUTO POSTO CHAVES LTDA                                    170.02
                                                                                    AUTO POSTO CINCO ESTRELAS LTDA.                           180.00
                                                                                    AUTO POSTO JARDIM BELVEDERE LTDA.                         193.00
                                                                                    BORGES CAHER COM. DE DERIVADOS DE PETROLEO LTDA           177.00
                                                                                    BRASAL COMBUSTÍVEIS LTDA.                                 150.00
                                                                                    CASCOL COMBUSTÍVEIS                                      3986.20
                                                                                    GOMES E GONTIJO LTDA                                    13000.00
                                                                                    JRM COMERCIO DE DERIVADO DE PETROLEO                      100.00
                                                                                    PETROIL COMBUSTÍVEIS LTDA.                                 20.00
                                                                                    PIONEIRA COMBUSTIVEIS LTDA                               4485.84
                                                                                    POSTO  VIA ESTRUTURAL                                      50.00
                                                                                    POSTO DA TORRE EIRELI EPP                                  85.00
                                                                                    POSTO DISBRAVE IMPERIAL LTDA.                             641.00
                                                                                    POSTO SABADIN LTDA                                         40.00
                                 CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.      ASPAR CENTRO DE SERVIÇOS PARLAMENTARES E GESTÃO...      36000.00
                                                                                    EFZ COMERCIO E REPRESENTAÇÃO LTDA                       75000.00
                                                                                    F. DA S. RABELO EPP                                    120000.00
                                 DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.               UP COMÉRCIO E INDÚSTRIA GRÁFICA LTDA                    80000.00
                                 EMISSÃO BILHETE AÉREO                              AZUL LINHAS AÉREAS                                       2027.18
                                                                                    GOL LINHAS AÉREAS                                       48052.44
                                                                                    LATAM                                                   24984.41
                                 FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR         DON´DURICA CAFE E RESTAURANTE LTDA EPP                     62.69
                                                                                    GM RIBEIRO DA COSTA RESTAURANTE - EPP                     134.00
                                 LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES               C.P.M. TRANSPORTE                                        9500.00
                                 LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES      ASSPAR-CRACIA CONSULTORIA LTD

In [5]:
base.pivot_table(index=['txNomeParlamentar','sgUF', 'sgPartido', 'txtDescricao', 'txtFornecedor', 'txtCNPJCPF'], values=['vlrDocumento'], aggfunc='sum').to_csv('deputados.csv')

In [20]:
base.pivot_table(index=['txtDescricao', 'txtCNPJCPF', 'txtFornecedorConsolidado', 'txNomeParlamentar'], values=['vlrDocumento'], aggfunc='sum')

vlrDocumento
txtDescricao              txtCNPJCPF   txtFornecedorConsolidado                           txNomeParlamentar                     
ASSINATURA DE PUBLICAÇÕES 1.172000e+09 DIÁRIOS ASSOCIADOS                                 ALESSANDRO MOLON                358.42
                                                                                          RODRIGO DE CASTRO               662.14
                                                                                          SARNEY FILHO                   1242.97
                          1.388000e+09 DISTRIBUIDORA BRASILIA DE VEICULOS S/A - POSTO ... SILAS BRASILEIRO                117.84
                          9.762600e+10 BRASAL COMBUSTÍVEIS LTDA.                          EVAIR DE MELO                   133.23
                                                                                          SILAS CÂMARA                    150.00
                          3.065970e+11 CASCOL COMBUSTÍVEIS                                ROBERTO GÓES                    120.00
                                                                                          IRACEMA PORTELLA                183.48
                                                                                          SILAS CÂMARA                    350.00
                                                                                          CHICO ALENCAR                   175.89
                                                                                          CELSO PANSERA                   135.02
                                                                                          JHC                              50.00
                          3.619450e+11 SISTEMA MEIO NORTE DE COMUNICACAO LTDA             ASSIS CARVALHO                  240.00
                                                                                          HERÁCLITO FORTES                881.44
                                                                                          JÚLIO CESAR                    2280.00
                          4.242750e+11 UNIGRAF UNIDAS GRAFICAS E EDITORA LTDA             ALEXANDRE BALDY                 456.00
                                                                                          JOÃO CAMPOS                     456.00
                                                                                          LUCAS VERGILIO                  456.00
                                                                                          MAGDA MOFATTO                   912.00
                          4.818410e+11 EDITORA NOTÍCIAS DO DIA LTDA                       MARCO TEBALDI                   258.00
                                                                                          MAURO MARIANI                   231.00
                          5.974910e+11 TRES COMERCIO DE PUBLICAÇÕES LTDA                  ALESSANDRO MOLON                854.00
                                                                                          LUIZIANNE LINS                 1598.00
                                                                                          MILTON MONTI                    721.00
                          6.455140e+11 POSTO CHAME-CHAME COMER. E SERV. DE COMBUST. LTDA  LUCIO VIEIRA LIMA                50.00
                          9.981210e+11 KEPLER COMERCIO E SERVIÇOS LTDA                    SERGIO SOUZA                    192.23
                          1.114830e+12 NOVOESTE - EDITORA JORNALISTICA NOVOESTE           CELSO MALDANER                  456.14
                          1.536754e+12 J.CAMARA & IRMÃOS S/A                              ALEXANDRE BALDY                 570.00
                                                                                          JOÃO CAMPOS                     543.60
                                                                                          LUCAS VERGILIO             

In [10]:
base.pivot_table(index=['txtDescricao', 'txtCNPJCPF', 'txtFornecedor', 'txNomeParlamentar'], values=['vlrDocumento'], aggfunc='sum').to_csv('fornecedores.csv')

In [5]:
from pandas import ExcelWriter
import openpyxl

writer = ExcelWriter('deputados_2015_fornecedor_consolidado.xlsx')
df1.to_excel(writer,'Sheet1')
writer.save()

ImportError: No module named openpyxl